In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import time

In [2]:
from state_space_setup import *


maximum_parts =  21


print("Length of the state space generated with max_parts =", maximum_parts, 
      "is:", len(get_state_space(maximum_parts)))
print("Index for (2, 3):", get_index(2, 3, maximum_parts))
print("Action space for state (2, 3):", get_action_space(np.array([2, 3]), maximum_parts))

Length of the state space generated with max_parts = 21 is: 253
Index for (2, 3): 46
Action space for state (2, 3): [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


In [3]:
def transition_probabilities(old_state, new_state, action):
    """
    Input: old_state (np.array): The previous state of the system.
           new_state (np.array): The potential next state of the system.
           action (int): The action taken, representing the number of parts ordered.
    Output: prob (float): The probability of transitioning from old_state to new_state given the action.

    This function calculates the transition probability from one state to another given an action.
    It is important to note that this function assumes the old_state and action are valid to provide
    more general applicability. The transition probabilities are base on three assumputions:
    1. Parts break down as a Poisson process (truncated at 0) with parameter lambda_
    2. The probability of the outstanding parts being delivered is fixed at p for any order size
    3. The newly ordered parts are added to the current outstaning order immediately.
    """
    # Parameters to set
    lambda_ = 2  # Average number of parts breaking down per week
    p = 0.9  # Probability of receiving the ordered parts

    if new_state[1] == 0:
        if (old_state[1] == 0) and (action == 0):
            prob = 1
        else:
            prob = p

        if new_state[0] <= (old_state[0] + old_state[1] + action) and new_state[0] > (old_state[1] + action):
            prob *= stats.poisson.pmf(old_state[0] + old_state[1] + action - new_state[0], mu=lambda_)
        elif new_state[0] == (old_state[1] + action):
            prob *= stats.poisson.sf(old_state[0] - 1, mu=lambda_)
        else:
            prob = 0
    elif new_state[1] == (old_state[1] + action):
        prob = 1 - p
        if new_state[0] <= old_state[0] and new_state[0] > 0:
            prob *= stats.poisson.pmf(old_state[0] - new_state[0],mu=lambda_)
        elif new_state[0] == 0:
            prob *= stats.poisson.sf(old_state[0] - 1, mu=lambda_)
        else:
            prob = 0
    else:
        prob = 0
    return prob


state_1 = np.array([4, 3])
state_2 = np.array([1, 3])
state_3 = np.array([4, 3])
action_1 = 3
action_2 = 0

print("Transition probability from state", state_1, "to", state_3, "with action", action_1, "is:",
      transition_probabilities(state_1, state_3, action_1))
print("Transition probability from state", state_1, "to", state_2, "with action", action_2, "is:",
      transition_probabilities(state_1, state_2, action_2))

Transition probability from state [4 3] to [4 3] with action 3 is: 0
Transition probability from state [4 3] to [1 3] with action 0 is: 0.01804470443154835


In [4]:
def downtime_cost(num_parts, k=168, lambda_=2, cost_per_week=16800):
    """
    Input: num_parts (int): The number of parts currently in the inventory.
           k (int): The number of periods in a week (default is 168 hours).
           lambda_ (float): The average number of parts breaking down per week (default is 2).
           cost_per_week (float): The cost incurred if the machine is not running for 
                                  an entire week (default is 16800).
    Output: cost (float): The cost incurred if shortage occurs in the next period for the 
                          given initial inventory level.

    This function calculates the shortage cost for a given state if shortage occurs in 
    the next period. Based on the model assumption - parts breaking as a Poisson process - 
    the shortage cost is calculated so that we get the same expected shortage cost as if we 
    assumed we split the week into k periods and penalise according to the number of periods the 
    machine was out of order.
    """
    if num_parts == 0:
        return cost_per_week

    # vector of numbers of periods of downtime, from 1 to k (e.g., 168 hours a week)
    m = np.arange(1, k + 1)
    # vector of numbers of parts broken before the period in which the last part breaks
    i = np.arange(num_parts)
    # vector of expected number of parts breaking down over periods 1 to k - m for each m
    lambda_m = (lambda_ / k) * (k - m[:, None])  

    # matrix of probabilities for each number of parts breaking down before the period 
    # in which the last part breaks for each such period possible
    pmf = stats.poisson.pmf(i[None, :], lambda_m)
    # vector of probabilities of all the remaining parts breaking down in the next period
    # for each possible number of parts broken before the period in which the last part breaks
    sf = stats.poisson.sf(num_parts - i - 1, lambda_ / k)

    # vector of probabilities of exactly m periods of downtime calculated using the above
    prob = np.sum(pmf * sf, axis=1)
    # weighted cost for each period of downtime, where m/k is the fraction of the week
    weighted_cost = prob * (m / k) * cost_per_week
    # total expected cost as the sum of weighted costs
    total_cost = np.sum(weighted_cost)
    # normalisation by the probability of running out of parts by next week, to obtain
    # the correct expected cost when multiplying by the probability of running out of parts
    normalisation = stats.poisson.sf(num_parts - 1, lambda_)

    return total_cost / normalisation


def cost_function(old_state, new_state, action):
    """
    Input: old_state (np.array): The previous state of the system.
           action (int): The action taken, representing the number of parts ordered.
           new_state (np.array): The potential next state of the system.
    Output: cost (float): The cost incurred by taking the action in the old state and transitioning to the new state.

    This function calculates the cost incurred by taking an action in a given state and transitioning to a new state.
    The cost is based on the number of parts ordered and the inventory levels before and after the transition.
    """
    holding_cost_as_percentage = 0.008  # percentage of the cost of a part to be paid for holding it in inventory
    ordering_cost = 200  # fixed cost per order placed
    price_per_part = 100  # price per part bought

    # calculate the total cost of the order made
    if action == 0:
        order_cost = 0
    else:
        order_cost = ordering_cost + price_per_part * action

    # calculate the shortage cost if the new state has only the newly arrived parts
    if (new_state[0] == (old_state[1] + action) and new_state[1] == 0) or (new_state[0] == 0):
        shortage_cost = downtime_cost(old_state[0])
    else:
        shortage_cost = 0

    # calculate holding cost for the new inventory level (paying for both delivered and ordered)
    holding_cost = holding_cost_as_percentage * price_per_part * (new_state[0] + new_state[1])

    return holding_cost + order_cost + shortage_cost


state_1 = np.array([4, 3])
state_2 = np.array([1, 3])
state_3 = np.array([3, 3])
action_1 = 3
action_2 = 0

print("Cost of transitioning from state", state_1, "to", state_1, "with action", action_1, "is:",
      cost_function(state_1, state_1, action_1))
print("Cost of transitioning from state", state_1, "to", state_2, "with action", action_2, "is:",
      cost_function(state_1, state_2, action_2))
print("Cost of transitioning from state", state_1, "to", state_3, "with action", action_1, "is:",
      cost_function(state_1, state_3, action_1))

print("Downtime cost for 4 parts in the initial inventory is:",
      downtime_cost(4))

Cost of transitioning from state [4 3] to [4 3] with action 3 is: 505.6
Cost of transitioning from state [4 3] to [1 3] with action 0 is: 3.2
Cost of transitioning from state [4 3] to [3 3] with action 3 is: 504.8
Downtime cost for 4 parts in the initial inventory is: 4467.816840972224


In [5]:
maximum_parts =  21
state_space = get_state_space(maximum_parts)


# generate the transition probability matrix
trans_prob_matrix = np.zeros((len(state_space), len(state_space), maximum_parts + 1))
# iterate over all old and new states and actions to fill the transition probability matrix
for i, s_old in enumerate(state_space):
    for j, s_new in enumerate(state_space):
        for a in range(maximum_parts + 1):
            trans_prob_matrix[i, j, a] = transition_probabilities(s_old, s_new, a)

# check if the sum of probabilities for each old inventory level and action is 1
all_correct = True
for i, s_old in enumerate(state_space):
    for a in range(maximum_parts - np.sum(s_old) + 1):
        total_prob = round(np.sum(trans_prob_matrix[i, :, a]), 9)
        if total_prob != 1.0:
            print(f"Probability sum for old state {s_old}, and action {a}: {total_prob:.9f}")
            all_correct = False
if all_correct:
    print("All transition probabilities sum to 1 for each old state and action pair. Test passed.")
else:
    print("Some transition probabilities do not sum to 1, see above. Test failed.")


# repeat the same for the cost function
cost_matrix = np.zeros((len(state_space), len(state_space), maximum_parts + 1))
for i, s_old in enumerate(state_space):
    for j, s_new in enumerate(state_space):
        for a in range(maximum_parts + 1):
            cost_matrix[i, j, a] = cost_function(s_old, s_new, a)


# visual check of the cost matrix for a specific inital state 
print("Cost matrix for initial state (8,3):\n")
display(pd.DataFrame(cost_matrix[get_index(8, 3, maximum_parts), :, :], index=np.arange(len(state_space)), columns=np.arange(maximum_parts + 1)))


# save the transition probability and cost matrices to files
np.save('cost_matrix.npy', cost_matrix)
np.save('trans_prob_matrix.npy', trans_prob_matrix)

All transition probabilities sum to 1 for each old state and action pair. Test passed.
Cost matrix for initial state (8,3):



,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,2301.040832,2601.040832,2701.040832,2801.040832,2901.040832,3001.040832,3101.040832,3201.040832,3301.040832,3401.040832,...,3701.040832,3801.040832,3901.040832,4001.040832,4101.040832,4201.040832,4301.040832,4401.040832,4501.040832,4601.040832
1,2301.840832,2601.840832,2701.840832,2801.840832,2901.840832,3001.840832,3101.840832,3201.840832,3301.840832,3401.840832,...,3701.840832,3801.840832,3901.840832,4001.840832,4101.840832,4201.840832,4301.840832,4401.840832,4501.840832,4601.840832
2,2302.640832,2602.640832,2702.640832,2802.640832,2902.640832,3002.640832,3102.640832,3202.640832,3302.640832,3402.640832,...,3702.640832,3802.640832,3902.640832,4002.640832,4102.640832,4202.640832,4302.640832,4402.640832,4502.640832,4602.640832
3,2303.440832,2603.440832,2703.440832,2803.440832,2903.440832,3003.440832,3103.440832,3203.440832,3303.440832,3403.440832,...,3703.440832,3803.440832,3903.440832,4003.440832,4103.440832,4203.440832,4303.440832,4403.440832,4503.440832,4603.440832
4,2304.240832,2604.240832,2704.240832,2804.240832,2904.240832,3004.240832,3104.240832,3204.240832,3304.240832,3404.240832,...,3704.240832,3804.240832,3904.240832,4004.240832,4104.240832,4204.240832,4304.240832,4404.240832,4504.240832,4604.240832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,16.000000,316.000000,416.000000,516.000000,616.000000,716.000000,816.000000,916.000000,1016.000000,1116.000000,...,1416.000000,1516.000000,1616.000000,1716.000000,1816.000000,1916.000000,2016.000000,2116.000000,2216.000000,2316.000000
249,16.800000,316.800000,416.800000,516.800000,616.800000,716.800000,816.800000,916.800000,1016.800000,1116.800000,...,1416.800000,1516.800000,1616.800000,1716.800000,1816.800000,1916.800000,2016.800000,2116.800000,2216.800000,2316.800000
250,16.000000,316.000000,416.000000,516.000000,616.000000,716.000000,816.000000,916.000000,1016.000000,1116.000000,...,1416.000000,1516.000000,1616.000000,1716.000000,1816.000000,4217.040832,2016.000000,2116.000000,2216.000000,2316.000000
251,16.800000,316.800000,416.800000,516.800000,616.800000,716.800000,816.800000,916.800000,1016.800000,1116.800000,...,1416.800000,1516.800000,1616.800000,1716.800000,1816.800000,1916.800000,2016.800000,2116.800000,2216.800000,2316.800000


In [6]:
def expected_action_value(current_state, action, values):
    """
    Input: current_state (tuple): The current state of the system.
           action (int): The action taken, representing the number of parts ordered.
           values (np.array): The value function for each state.
    Output: expected_value (float): The expected value of taking the action in the current state.
    """
    # Using global variables for the matrices and the maximum number of parts
    global maximum_parts
    global cost_matrix
    global trans_prob_matrix

    discount_factor = 0.995  # discount factor for future rewards

    # get the index of the current state in the state space
    current_state_idx = get_index(current_state[0], current_state[1], maximum_parts)

    probs = trans_prob_matrix[current_state_idx, :, action]
    costs = cost_matrix[current_state_idx, :, action]
    expected_value = np.dot(probs, costs + discount_factor * values)
    return expected_value


# testing the expected_action_value
current_state = (5, 2)
action = 3

expected_value = expected_action_value(current_state, action, values=np.zeros(len(state_space)))
print(f"Expected value for action {action} in state {current_state}: {expected_value}\n")

# convert the 2D numpy array into a list of tuples for better readability
state_tuples = list(map(tuple, state_space))
# get current state index
current_idx = get_index(current_state[0], current_state[1], maximum_parts)
check_df = pd.DataFrame({
    "Next State": state_tuples,
    "Transition Probability": trans_prob_matrix[current_idx, :, action],
    "Cost": cost_matrix[current_idx, :, action]
})
display(check_df[check_df["Transition Probability"] > 0].sort_values(by="Transition Probability", ascending=False))

Expected value for action 3 in state (5, 2): 697.9587271957992



,Next State,Transition Probability,Cost
148,"(8, 0)",0.243604,506.400000
162,"(9, 0)",0.243604,507.200000
133,"(7, 0)",0.162402,505.600000
175,"(10, 0)",0.121802,508.000000
117,"(6, 0)",0.081201,504.800000
100,"(5, 0)",0.047388,4141.792219
68,"(3, 5)",0.027067,506.400000
87,"(4, 5)",0.027067,507.200000
48,"(2, 5)",0.018045,505.600000
105,"(5, 5)",0.013534,508.000000


In [7]:
# Value iteration
error = 1e-6
values = np.zeros(len(state_space))
delta = 2 * error
counter = 0
elapsed = 0

while delta >= error:
    start_time = time.time()
    delta = 0
    old_values = np.copy(values)
    for i, state in enumerate(state_space):
        v = values[i]
        action_space = get_action_space(state, maximum_parts)
        action_costs = np.array([
            expected_action_value(state, act, old_values)
            for act in action_space
        ])
        values[i] = np.min(action_costs)
        delta = max(delta, abs(v - values[i]))
    counter += 1
    end_time = time.time()
    elapsed += end_time - start_time
    if counter % 400 == 0:
        avg_time = elapsed / 400
        print(f"Finished {counter} iterations \nCurrent delta = {delta}\n")
        print(f"{avg_time:.4f} seconds per iteration")
        print(values)
        print()
        elapsed = 0

print(values)

Finished 400 iterations 
Current delta = 34.784805304756446

0.0066 seconds per iteration
[63589.61169821 63489.61169821 63389.61169821 63289.61169821
 63189.61169821 63089.61169821 62989.61169821 62889.61169821
 62789.61169821 62689.61169821 62585.75755772 62480.15328977
 62353.06667862 62230.88230382 62111.05733869 61991.74689303
 61872.98833421 61754.9158644  61637.54949451 61520.87954074
 61404.90082342 61289.61169821 56272.07158234 56172.07158234
 56072.07158234 55972.07158234 55872.07158234 55772.07158234
 55672.07158234 55572.07158234 55472.07158234 55365.94913483
 55254.12471513 55145.86667105 55019.38564736 54897.49273728
 54777.73133223 54658.48906275 54539.8152585  54421.83001526
 54304.54967567 54187.96511234 54072.07158234 50973.62443632
 50873.62443632 50773.62443632 50673.62443632 50573.62443632
 50473.62443632 50373.62443632 50273.62443632 50173.62443632
 50053.62569435 49940.92481952 49827.8923643  49702.93051241
 49581.69204863 49462.12767508 49343.10837369 49224.6936

In [8]:
state_tuples = list(map(tuple, state_space))

policy_records = []
for state in state_tuples:
    action_space = get_action_space(state, maximum_parts)
    action_costs = np.array([
        expected_action_value(state, act, values)
        for act in action_space
    ])
    order = action_space[np.argmin(action_costs)]
    policy_records.append((state, order))

policy_df = pd.DataFrame(policy_records, columns=["State", "Order_size"])
display(policy_df)
policy_df.to_csv("policy.csv", index=False)

,State,Order_size
0,"(0, 0)",21
1,"(0, 1)",20
2,"(0, 2)",19
3,"(0, 3)",18
4,"(0, 4)",17
...,...,...
248,"(19, 1)",0
249,"(19, 2)",0
250,"(20, 0)",0
251,"(20, 1)",0


This approximately gives an (s,S) policy with $s = 12$ and $S = 21$ where the current inventory position is the variable of interest. However, for inventory level less than or equal to 1 we get $s = 11$.